
---

# 📂 5. Docker Volumes & Bind Mounts

## A) What & Why

* **Volumes** = persistent data managed by Docker (survive container removal).
* **Bind mounts** = map **host folder ↔ container** (great for live dev).

---

## B) Types (when to use)

* **Anonymous volume**: auto-named; quick/throwaway.
* **Named volume**: reusable across containers; **prod default**.
* **Bind mount**: host path → container path; **dev/workspace**.

---

## C) Core volume commands

```bash
docker volume create myvol        # make
docker volume ls                  # list
docker volume inspect myvol       # details
docker volume rm myvol            # delete one
docker volume prune               # delete unused
```

---

## D) Use in `docker run`

**Short `-v` form**

```bash
# named volume → persistent DB data
docker run -d -v myvol:/var/lib/postgresql/data postgres

# bind mount current dir → /app (live edits)
docker run -it -v "$(pwd)":/app node:18 bash

# read-only bind mount
docker run -d -v "$(pwd)"/nginx.conf:/etc/nginx/nginx.conf:ro nginx
```

**Explicit `--mount` form (clearer)**

```bash
docker run -d --mount type=volume,src=myvol,dst=/data busybox
docker run -d --mount type=bind,src="$(pwd)"/site,dst=/usr/share/nginx/html,ro nginx
```

---

## E) Quick recipes

```bash
# share a volume between two containers
docker run -d --name db -v myvol:/data busybox
docker run -it --name job --volumes-from db busybox sh

# backup a named volume to host tar
docker run --rm -v myvol:/data -v "$(pwd)":/backup busybox \
  sh -c 'cd /data && tar cf /backup/myvol.tar .'

# restore into a fresh volume
docker volume create myvol2
docker run --rm -v myvol2:/data -v "$(pwd)":/backup busybox \
  sh -c 'cd /data && tar xf /backup/myvol.tar'
```

---

## F) Best practices (1-liners)

* **Prod:** prefer **named volumes**; **Dev:** bind mounts.
* Don’t mount over needed init files (it **hides** image content at that path).
* Watch **permissions/UID** (use non-root or `chown` in Dockerfile).
* Prune safely: `docker system prune` **does not** remove volumes; use `docker volume prune`.
